<a href="https://colab.research.google.com/github/christinium/japan2019/blob/master/5%20-%20Pneumothorax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pneumothorax example

## Sentence tokenization, and spotting term + negation

This example spots meantions of the "pneumothorax" lexicon in CXR reports and looks at whether the spotted pneumothorax mentioned was negated or not. 

Originally contributed by:

*Joy Wu* <joy.wu@ibm.com>*, *Daniel Gruhl <dgruhl@us.ibm.com>*

In [0]:
# Required files
import requests
from requests.auth import HTTPBasicAuth
import json
import matplotlib.pyplot as plt
import subprocess
import tempfile
import os
import sys
import pandas as pd
import numpy as np
import re
import random
from nltk.tokenize import sent_tokenize



from google.colab import auth
from google.cloud import bigquery
from google.colab import files

**Authenticate:** The line of code below ensures you are an authenticated user accessing the MIMIC database. You will need to rerun this each time you open the notebook.

In [0]:
auth.authenticate_user() #This will allow you to authenticate access to BigQuery

**Query Function: **This is a method that executes a desired SQL query on the database. If you want to run a query, you can use the function name below, which we named run_query()


In [0]:
project_id='datathonjapan2019'
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id
# Read data from BigQuery into pandas dataframes.
def run_query(query):
  return pd.io.gbq.read_gbq(query, project_id=project_id, verbose=False, configuration={'query':{'useLegacySql': False}})

### Sentence splitting:

In [5]:
# Read the sample CXR reports 

CXRreports = run_query('''
SELECT * 
FROM `physionet-data.mimiciii_nlp_demo.cxr`
''')
CXRreports.head()


,row_id,subject_id,hadm_id,chartdate,charttime,storetime,category,description,cgid,iserror,text
0,739780,24248,None,2158-05-23,2158-05-23 16:00:00,,Radiology,CHEST (PA & LAT),,,[**2158-5-23**] 4:00 PM\n CHEST (PA & LAT) ...
1,739802,15017,None,2152-07-20,2152-07-20 10:54:00,,Radiology,CHEST (PA & LAT),,,[**2152-7-20**] 10:54 AM\n CHEST (PA & LAT) ...
2,739893,20979,None,2168-08-19,2168-08-19 17:32:00,,Radiology,CHEST (PA & LAT),,,[**2168-8-19**] 5:32 PM\n CHEST (PA & LAT) ...
3,739863,22853,None,2155-07-29,2155-07-29 14:02:00,,Radiology,CHEST (PA & LAT),,,[**2155-7-29**] 2:02 PM\n CHEST (PA & LAT) ...
4,739757,15374,None,2133-05-18,2133-05-18 10:53:00,,Radiology,CHEST (PA & LAT),,,[**2133-5-18**] 10:53 AM\n CHEST (PA & LAT) ...


In [6]:
#This prints the first report (numbering starts at 0). You can change the report by changing "0" to another number
report = CXRreports.text[0]
print(report)

[**2158-5-23**] 4:00 PM
 CHEST (PA & LAT)                                                Clip # [**Clip Number (Radiology) 50638**]
 Reason: follow up cxr for pt with pneumomia rml
 ______________________________________________________________________________
 [**Hospital 2**] MEDICAL CONDITION:
  37 year old man with sob, chest and left LE DVT
 REASON FOR THIS EXAMINATION:
  follow up cxr for pt with pneumomia rml
 ______________________________________________________________________________
                                 FINAL REPORT
 INDICATION:  Right middle lobe pneumonia.

 CHEST TWO VIEWS:  There are consolidation/collapse in the right middle lobe
 and in the anterior segment of the right lower lobe, not significantly changed
 since [**2158-5-16**].  There is minimal linear atelectasis at the left lung base.  A
 small right pleural effusion is present.  The left costophrenic angle is
 clear.  Pulmonary vascularity is normal.  Cardiomediastinal silhouette is
 normal.

 IMPRES

In [7]:
  #This imports nltk and punkt into our environment
  >>> import nltk
  >>> nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
# Tokenize the sentences with sent_tokenize from NLTK
from nltk.tokenize.punkt import PunktSentenceTokenizer
# Alternatively, tokenize with PunktSentenceTokenizer from NLTK if you want to keep track of character offsets of sentences
sents = sent_tokenize(report.replace('\n',' ')) # removing new line breaks
# Print out list of sentences:
sent_count = 0
for s in sents:
    print("Sentence " + str(sent_count) +":")
    print(s)
    print()
    sent_count = sent_count + 1

Sentence 0:
[**2158-5-23**] 4:00 PM  CHEST (PA & LAT)                                                Clip # [**Clip Number (Radiology) 50638**]  Reason: follow up cxr for pt with pneumomia rml  ______________________________________________________________________________  [**Hospital 2**] MEDICAL CONDITION:   37 year old man with sob, chest and left LE DVT  REASON FOR THIS EXAMINATION:   follow up cxr for pt with pneumomia rml  ______________________________________________________________________________                                  FINAL REPORT  INDICATION:  Right middle lobe pneumonia.

Sentence 1:
CHEST TWO VIEWS:  There are consolidation/collapse in the right middle lobe  and in the anterior segment of the right lower lobe, not significantly changed  since [**2158-5-16**].

Sentence 2:
There is minimal linear atelectasis at the left lung base.

Sentence 3:
A  small right pleural effusion is present.

Sentence 4:
The left costophrenic angle is  clear.

Sentence 5:
Pulmonary va

In [9]:
sent_count = 0
for s_start, s_finish in PunktSentenceTokenizer().span_tokenize(report):
    print("Sentence " + str(sent_count) +": " + str([s_start, s_finish]))
    print(report[s_start:s_finish].replace('\n',' '))
    print()
    sent_count = sent_count + 1

Sentence 0: [0, 588]
[**2158-5-23**] 4:00 PM  CHEST (PA & LAT)                                                Clip # [**Clip Number (Radiology) 50638**]  Reason: follow up cxr for pt with pneumomia rml  ______________________________________________________________________________  [**Hospital 2**] MEDICAL CONDITION:   37 year old man with sob, chest and left LE DVT  REASON FOR THIS EXAMINATION:   follow up cxr for pt with pneumomia rml  ______________________________________________________________________________                                  FINAL REPORT  INDICATION:  Right middle lobe pneumonia.

Sentence 1: [591, 770]
CHEST TWO VIEWS:  There are consolidation/collapse in the right middle lobe  and in the anterior segment of the right lower lobe, not significantly changed  since [**2158-5-16**].

Sentence 2: [772, 830]
There is minimal linear atelectasis at the left lung base.

Sentence 3: [832, 875]
A  small right pleural effusion is present.

Sentence 4: [877, 915]
The left co

### Spot occurrence(s) of word(s) related to your concept in a sentence or document

In [0]:
# Simple spotter: Spot occurrence of a term in a given lexicon anywhere within a text document or sentence:
def spotter(text, lexicon):
    text = text.lower()
    # Spot if a document mentions any of the terms in the lexicon (not worrying about negation detection yet)
    match = [x in text for x in lexicon]
    if any(match) == True:
        mentioned = 1
    else:
        mentioned = 0
    return mentioned

In [0]:
# Where the lexicon is a list of word(s) or phrase(s) refering to a concept of interest to you, e.g.
ptx = ['pneumothorax', 'ptx']
sent1 = 'Large left apical ptx present.'
sent2 = 'Hello world for NLP'


In [12]:
# lexicon mentioned in text, spotter return 1 (yes)
spotter(sent1, ptx)

1

In [13]:
# lexicon not mentioned in text, spotter return 0 (no)
spotter(sent2, ptx)

0

**How can we do better?**
We can do the spotting of concepts (lexicons) A LOT better (more sensitive) if we curate a list of all the ways that the concept could be expressed in raw text. This is what the NLP tool can help you achieve.

### Lexicons - there are tools out there where you can get lexicons.  Using an IBM one, we have a list of terms listed below

In [14]:
# Printing out the pneumothorax lexicon (after 5 minutes of curating work on the NLP tool)
ptx = ['pneumothorax', 'ptx', 'pneumothoraces', 'pnuemothorax', 'pnumothorax', 'pntx', 'penumothorax', 'pneomothorax', 'pneumonthorax', 'pnemothorax', 'pneumothoraxes', 'pneumpthorax', 'pneuomthorax', 'pneumothorx', 'pneumothrax', 'pneumothroax', 'pneumothraces', 'pneunothorax', 'enlarging pneumo', 'pneumothoroax', 'pneuothorax']
print(ptx)

['pneumothorax', 'ptx', 'pneumothoraces', 'pnuemothorax', 'pnumothorax', 'pntx', 'penumothorax', 'pneomothorax', 'pneumonthorax', 'pnemothorax', 'pneumothoraxes', 'pneumpthorax', 'pneuomthorax', 'pneumothorx', 'pneumothrax', 'pneumothroax', 'pneumothraces', 'pneunothorax', 'enlarging pneumo', 'pneumothoroax', 'pneuothorax']


### Negation detection

In [15]:
# But it's not enough to just spot word occurrences to determine if a concept is affirmative (positive/present) or not.

# e.g. lexicon mentioned in text but negated, a simple spotter would still return 1 (yes)
sent3 = 'Pneumothorax has resolved.'
spotter(sent3, ptx)

1

In [16]:
# However, if negation related words occur in close proximity (e.g. same sentence) to a spotted concept 
# Then we can right some rules to determine if the concept was negated or not

# e.g. spotting negation words in the same sentence:
neg = ['no','never','not','removed', 'ruled out', 'resolved']
spotter(sent3, neg)

1

### Using off-the-shelf python library for negation, e.g. Negex

In [17]:
#This downloads a copy of negex.py and negex_triggers.txt into this environment, we will learn how to use this in the next block
!wget  https://stuff.mit.edu/~cwc76/hst953/negex.py
!wget  https://stuff.mit.edu/~cwc76/hst953/negex_triggers.txt
  


--2019-03-05 19:00:07--  https://stuff.mit.edu/~cwc76/hst953/negex.py
Resolving stuff.mit.edu (stuff.mit.edu)... 18.4.60.31
Connecting to stuff.mit.edu (stuff.mit.edu)|18.4.60.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8984 (8.8K) [text/x-python]
Saving to: ‘negex.py’

negex.py            100%[===================>]   8.77K  --.-KB/s    in 0s      

2019-03-05 19:00:07 (56.5 MB/s) - ‘negex.py’ saved [8984/8984]

--2019-03-05 19:00:11--  https://stuff.mit.edu/~cwc76/hst953/negex_triggers.txt
Resolving stuff.mit.edu (stuff.mit.edu)... 18.4.60.31
Connecting to stuff.mit.edu (stuff.mit.edu)|18.4.60.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7363 (7.2K) [text/plain]
Saving to: ‘negex_triggers.txt’

negex_triggers.txt  100%[===================>]   7.19K  --.-KB/s    in 0s      

2019-03-05 19:00:11 (171 MB/s) - ‘negex_triggers.txt’ saved [7363/7363]



In [18]:
import negex
rfile = open(r'negex_triggers.txt')
irules = negex.sortRules(rfile.readlines())
rfile.close()

# Example:
sent = "There is no evidence of ptx."
ptx = ['pneumothorax', 'ptx']
tagger = negex.negTagger(sentence = sent, phrases = ptx, rules = irules, negP=False)
negation = tagger.getNegationFlag()
negation

'negated'

In [0]:
# Applying Negex to the first note:
note = CXRreports['text'][16]
# Tokenize the sentences:
sents_blob = sent_tokenize(note.replace('\n',' ')) # replacing new line breaks with a space, splits into sentences
# Applying spotter function to each sentence:
#neg_output = []
count = 0
for sent in sents:
    # Apply Negex if a term in the ptx lexicon is spotted
    if spotter(sent,ptx) == 1:
        tagger = negex.negTagger(sentence = sent, phrases = ptx, rules = irules, negP=False)
        negation = tagger.getNegationFlag()
        #neg_output.append(negation)
        print("Sentence " + str(count) + ":\n" + sent + "\nNegex output: " + negation + '\n')
        count = count + 1

In [20]:
# Show the relevant CXR report for the analysis
print(note)

[**2168-6-12**] 4:05 AM
 CHEST (PORTABLE AP)                                             Clip # [**Clip Number (Radiology) 102597**]
 Reason: reeval ETT placement
 ______________________________________________________________________________
 [**Hospital 2**] MEDICAL CONDITION:
  27 year old man intubated for airway protection and drugs -v- EtOH. ETT very
  distal and repositioned. eval placement
 REASON FOR THIS EXAMINATION:
  reeval ETT placement
 ______________________________________________________________________________
                                 FINAL REPORT
 CLINICAL INDICATION: 27 year old man intubated for airway protection. Status
 post readjustment of endotracheal tube positioning. Please reevaluate.

 A single supine portable radiograph of the chest was obtained and compared to
 the next prior study of approximately 20 minutes earlier.

 Since the previous examination, the endotracheal tube has been advanced and
 its tip is now appropriately located approximately 5

### Exercise for you:

You can use similar/improved pipeline to loop through all the notes in your dataset and through different concepts/lexicons!